# __4.3 FastText__

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'ML/KRembedding/'

LIB_PATH = 'lib'
DATA_PATH = '/notebooks/embedding/data'

%cd drive/My\ Drive/$FOLDERNAME/

# 형태소 분석 완료된 데이터 다운로드
!bash preprocess.sh dump-tokenized

Mounted at /content/drive
/content/drive/My Drive/ML/KRembedding


In [ ]:
!cd /notebooks/embedding/data/tokenized && cat wiki_ko_mecab.txt ratings_mecab.txt korquad_mecab.txt > corpus_mecab.txt

In [5]:
# Mecab 설치
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Installing automake (A dependency for mecab-ko)
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.9 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/

In [5]:
%cd $LIB_PATH
import konlpy
import soynlp
import soyspacing
import sentencepiece
import fasttext
import khaiii
%cd ../

/content/drive/My Drive/ML/KRembedding/lib
/content/drive/My Drive/ML/KRembedding


In [ ]:
%cd /content
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText && make
%cd drive/My\ Drive/$FOLDERNAME/

/content
Cloning into 'fastText'...
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Receiving objects: 100% (3854/3854), 8.22 MiB | 15.98 MiB/s, done.
Resolving deltas: 100% (2417/2417), done.
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/args.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/autotune.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/matrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/dictionary.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/loss.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/productquantizer.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/densematrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/quantmatrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DND

In [ ]:
!mkdir -p $DATA_PATH/word-embeddings/fasttext
!/content/fastText/fasttext skipgram \
  -input $DATA_PATH/tokenized/corpus_mecab.txt \
  -output $DATA_PATH/word-embeddings/fasttext/fasttext

Read 233M words
Number of words:  358043
Number of labels: 0
Progress: 100.0% words/sec/thread:   11053 lr:  0.000000 avg.loss:  0.473699 ETA:   0h 0m 0s


In [3]:
!mkdir -p $DATA_PATH/word-embeddings/fasttext
!cp lib/fasttext/fasttext* $DATA_PATH/word-embeddings/fasttext

In [7]:
from models.word_eval import WordEmbeddingEvaluator
model = WordEmbeddingEvaluator(
    vecs_txt_fname=DATA_PATH+'/word-embeddings/fasttext/fasttext.vec',
    vecs_bin_fname=DATA_PATH+'/word-embeddings/fasttext/fasttext.bin',
    method='fasttext', dim=100, tokenizer_name='mecab'
)

In [8]:
import pandas as pd
pd.DataFrame({
    sent: [tu[0] for tu in model.most_similar(sent, topn=5)] \
    for sent in ['희망','절망','학교','학생','가족','자동차','하였다']
})

,희망,절망,학교,학생,가족,자동차,하였다
0,행복,절망감,초등,재학생,부부,상용차,하
1,소망,고뇌,중학교,대학생,편부모,경자동차,다
2,꿈,외로움,ㆍ중학교,교직원,부모,승용차,했
3,희망찬,상실감,분교,교내,입양아,안전자동차,였으며
4,기쁨,슬픔,개교,학부모,대가족,SUV,했으며


In [12]:
unk_word = '서울특벌시'
display(model._is_in_vocabulary(unk_word))
display(model.get_word_vector(unk_word))
display(model.most_similar(unk_word, topn=5))

False

array([-0.21241017,  0.11198233,  0.14365713, -0.22253615,  0.00329546,
        0.22972704, -0.2906415 ,  0.07143973,  0.25511533,  0.49814615,
        0.1577383 ,  0.01182862, -0.14366971, -0.02334673, -0.23549697,
        0.23037556,  0.17935182, -0.08553979,  0.10581421,  0.152494  ,
        0.12219895,  0.3012272 ,  0.14770731,  0.02614469,  0.05211823,
        0.26433384,  0.26215696, -0.26485538,  0.12543026,  0.22253865,
        0.31454077,  0.14063573, -0.03700489,  0.12421688, -0.19775753,
       -0.09605865, -0.14328699,  0.27043012, -0.14836691, -0.2760007 ,
        0.42357695,  0.05753992, -0.19041355,  0.3443407 ,  0.2499727 ,
        0.15807231, -0.01800553,  0.01744822, -0.16198455, -0.09302557,
       -0.21218656,  0.4993079 , -0.06491352,  0.19671029,  0.24820912,
       -0.27807865,  0.03181083, -0.25785396,  0.07925627,  0.06964847,
        0.13466814, -0.10277382,  0.03155977,  0.22883019,  0.13466863,
       -0.11836469, -0.20348893,  0.09860119, -0.09321549, -0.29

[('서울색', 0.7208898176319862),
 ('서울한강체', 0.6827552122661364),
 ('DONGGUK', 0.6626824525222298),
 ('등이', 0.6464735902274431),
 ('통색', 0.644664014356914)]

In [13]:
from preprocess import jamo_sentence, get_tokenizer
tokenizer = get_tokenizer('mecab')
tokens = ' '.join(tokenizer.morphs('나는 학교에 간다'))
print(jamo_sentence(tokens))

ㄴㅏ- ㄴㅡㄴ ㅎㅏㄱㄱㅛ- ㅇㅔ- ㄱㅏㄴㄷㅏ-


In [ ]:
!mkdir -p $DATA_PATH/word-embeddings/fasttext-jamo
!/content/fastText/fasttext skipgram \
  -input $DATA_PATH/tokenized/corpus_mecab_jamo.txt \
  -output $DATA_PATH/word-embeddings/fasttext-jamo/fasttext-jamo

In [ ]:
from models.word_eval import WordEmbeddingEvaluator
model_jamo = WordEmbeddingEvaluator(
    vecs_txt_fname=DATA_PATH+'word-embeddings/fasttext-jamo/fasttext-jamo.vec',
    vecs_bin_fname=DATA_PATH+'word-embeddings/fasttext-jamo/fasttext-jamo.bin',
    method='fasttext-jamo', dim=100, tokenizer_name='mecab'
)

In [ ]:
pd.DataFrame({
    sent: [tu[0] for tu in model_jamo.most_similar(sent, topn=5)] \
    for sent in ['희망','절망','학교','학생','가족','자동차','하였다',unk_word]
})